<a href="https://colab.research.google.com/github/onmax/bike-forecasting/blob/main/src/preprocessing/dataset-creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Getting the dataset
!git clone "https://github.com/onmax/bike-forecasting"
%cd bike-forecasting/src/preprocessing

Cloning into 'bike-forecasting'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.
Checking out files: 100% (35/35), done.
/content/bike-forecasting


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# files got from https://divvy-tripdata.s3.amazonaws.com/index.html. Data folder should look like this. Files folder contains the data from the zips
'''
├── files
│   └── {year}
│        └── *.csv
├── data
│   └── raw
│       ├── Divvy_Stations_Trips_2014_Q1Q2.zip
│       ├── Divvy_Stations_Trips_2014_Q3Q4.zip
│       ├── Divvy_Trips_2015-Q1Q2.zip
│       ├── Divvy_Trips_2015_Q3Q4.zip
│       ├── Divvy_Trips_2016_Q1Q2.zip
│       ├── Divvy_Trips_2016_Q3Q4.zip
│       ├── Divvy_Trips_2017_Q1Q2.zip
│       ├── Divvy_Trips_2017_Q3Q4.zip
│       ├── Divvy_Trips_2018_Q1.zip
│       ├── Divvy_Trips_2018_Q2.zip
│       ├── Divvy_Trips_2018_Q3.zip
│       ├── Divvy_Trips_2018_Q4.zip
│       ├── Divvy_Trips_2019_Q1.zip
│       ├── Divvy_Trips_2019_Q2.zip
│       ├── Divvy_Trips_2019_Q3.zip
│       └── Divvy_Trips_2019_Q4.zip
└── ...
'''

import pandas as pd
import pickle
import os

from preprocessing_lib import *


In [24]:
year = "2014"
csv_folder = path_data_for_year(year)
csv_names = ["Divvy_Trips_2014_Q1Q2.csv", "Divvy_Trips_2014-Q3-07.csv", "Divvy_Trips_2014-Q3-0809.csv", "Divvy_Trips_2014-Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["starttime", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/Divvy_Trips_2014_Q1Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/Divvy_Trips_2014-Q3-07.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/Divvy_Trips_2014-Q3-0809.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/Divvy_Trips_2014-Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/trips-2014.csv


In [32]:
year = "2015"
csv_folder = path_data_for_year(year)
csv_names = ["Divvy_Trips_2015-Q1.csv", "Divvy_Trips_2015-Q2.csv", "Divvy_Trips_2015_07.csv", "Divvy_Trips_2015_08.csv", "Divvy_Trips_2015_09.csv", "Divvy_Trips_2015_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["starttime", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015-Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015-Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015_07.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015_08.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015_09.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/trips-2015.csv


In [25]:
year = "2016"
csv_folder = path_data_for_year(year)
csv_names = ["Divvy_Trips_2016_Q1.csv", "Divvy_Trips_2016_04.csv", "Divvy_Trips_2016_05.csv", "Divvy_Trips_2016_06.csv","Divvy_Trips_2016_Q3.csv", "Divvy_Trips_2016_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["starttime", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_04.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_05.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_06.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_Q3.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/trips-2016.csv


In [26]:
year = "2017"
csv_folder = path_data_for_year(year)
csv_names = ["Divvy_Trips_2017_Q1.csv", "Divvy_Trips_2017_Q2.csv", "Divvy_Trips_2017_Q3.csv", "Divvy_Trips_2017_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["start_time", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/Divvy_Trips_2017_Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/Divvy_Trips_2017_Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/Divvy_Trips_2017_Q3.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/Divvy_Trips_2017_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/trips-2017.csv


In [27]:
year = "2018"
csv_folder = path_data_for_year(year)
csv_names = ["Divvy_Trips_2018_Q1.csv", "Divvy_Trips_2018_Q2.csv", "Divvy_Trips_2018_Q3.csv", "Divvy_Trips_2018_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["start_time", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/Divvy_Trips_2018_Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/Divvy_Trips_2018_Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/Divvy_Trips_2018_Q3.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/Divvy_Trips_2018_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/trips-2018.csv


In [28]:
year = "2019"
csv_folder = path_data_for_year(year)
csv_names = ["Divvy_Trips_2019_Q1.csv", "Divvy_Trips_2019_Q2.csv", "Divvy_Trips_2019_Q3.csv", "Divvy_Trips_2019_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["start_time", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/Divvy_Trips_2019_Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/Divvy_Trips_2019_Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/Divvy_Trips_2019_Q3.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/Divvy_Trips_2019_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/trips-2019.csv


In [41]:
# Merge all pickles
years = list(range(2014, 2020))
csvs = get_csvs(years)
df = merge_years(csvs)
df

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/trips-2014.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/trips-2015.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/trips-2016.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/trips-2017.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/trips-2018.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/trips-2019.csv


,from_station_id,from_station_name,start_time
0,131,Lincoln Ave & Belmont Ave,2014-06-30 23:57:00
1,282,Halsted St & Maxwell St,2014-06-30 23:56:00
2,327,Sheffield Ave & Webster Ave,2014-06-30 23:33:00
3,134,Peoria St & Jackson Blvd,2014-06-30 23:26:00
4,320,Loomis St & Lexington St,2014-06-30 23:16:00
...,...,...,...
3817999,112,Green St & Randolph St,2019-12-31 23:56:13
3818000,90,Millennium Park,2019-12-31 23:56:34
3818001,623,Michigan Ave & 8th St,2019-12-31 23:57:05
3818002,623,Michigan Ave & 8th St,2019-12-31 23:57:11


In [52]:
# Group trips by hour
df = nn_format(df)
df

,quantity_1,quantity_100,quantity_101,quantity_102,quantity_103,quantity_106,quantity_107,quantity_108,quantity_109,quantity_11,...,quantity_90,quantity_91,quantity_92,quantity_93,quantity_94,quantity_95,quantity_96,quantity_97,quantity_98,quantity_99
start_time,,,,,,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2014-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-31 20:00:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
save_with_split(df=df, write_size=49500000, dest_folder=f'{path()}/../parts')

Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-1
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-2
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-3


In [54]:
parts = [f"{path()}/../parts/chicago-divvy-trips-part-{i}" for i in list(range(1,4))]
join(f'{path()}/trips.csv', parts)

Putting together /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-1
Putting together /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-2
Putting together /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-3
